In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import sys, os
import torch.nn.functional as F
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
import transformers

class MLPModel(nn.Module):
    def __init__(self,fe,out_num):
        super(MLPModel, self).__init__()
        inp_num=len(fe)
        
        hid1=2048*2
        hid2=2048
        hid3=2048
        hid4=2048
        
        self.fc1 = nn.Linear(inp_num,hid1)
        self.fc2 = nn.Linear(hid1, hid2)
        self.fc3 = nn.Linear(hid2, hid3)
        self.fc4 = nn.Linear(hid3, hid4)
        
        self.output = nn.Linear(hid2,out_num)
        
        self.bn = nn.BatchNorm1d(inp_num)
        self.ln = nn.LayerNorm(inp_num)
        self.relu = nn.ReLU()

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        
#         self.fc1_list=nn.ModuleList([nn.Linear(hid_num,256) for i in range(10)])
#         self.fc2_list=nn.ModuleList([nn.Linear(256,256) for i in range(10)])
#         self.fc3_list=nn.ModuleList([nn.Linear(256,256) for i in range(10)])
#         self.output_list=nn.ModuleList([nn.Linear(256, len(label_1_name)+len(label_2_name)) for i in range(10)])
        
    def forward(self, X):
        #X=self.ln(X)
        X = self.fc1(X)
        X = self.relu(X)
        
        X = self.fc2(X)
        X = self.relu(X)
        
#         X = self.fc3(X)
#         X = self.relu(X)
        
#         X = self.fc4(X)
#         X = self.relu(X)
        
#         X = self.fc3(X)
#         #X = self.dropout2(X)
#         X = self.relu(X)
        preds = self.output(X)
        #print(preds.shape)
        return preds
    
############################################################################################################
############################################################################################################
############################################################################################################
    
train_peptides=pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
train_proteins=pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
Peptide=pd.DataFrame({"Peptide":list(train_peptides["Peptide"].unique())})
NPX=pd.DataFrame({"NPX":list(train_proteins["NPX"].unique())})

train_peptides["PeptideAbundance"]=train_peptides["PeptideAbundance"]/(train_peptides.groupby("visit_id").PeptideAbundance.transform("sum")+1)
train_proteins["NPX"]=train_proteins["NPX"]/(train_proteins.groupby("visit_id").NPX.transform("sum")+1)

train_proteins_pivot=train_proteins.pivot(index=["visit_id"],columns=["UniProt"],values=["NPX"]).reset_index()
train_proteins_pivot.columns=["_".join(i).strip("_") for i in train_proteins_pivot.columns]

train_peptides_pivot=train_peptides.pivot(index=["visit_id"],columns=["Peptide"],values=["PeptideAbundance"]).reset_index()
train_peptides_pivot.columns=["_".join(i).strip("_") for i in train_peptides_pivot.columns]


############################################################################################################
############################################################################################################
############################################################################################################


label_1_name=["label"]
label_2_name=["label_updrs_1","label_updrs_2","label_updrs_3","label_updrs_4"]
label_3_name=["label_0","label_6","label_12","label_24"]
label_4_name=[i+"_"+str(j) for i in ['updrs_1', 'updrs_2','updrs_3', 'updrs_4'] for j in [0,6,12,24]]

valid_month=[0, 6, 12, 18, 24 , 36, 48, 60, 72, 84]
p_features=list(train_proteins_pivot.columns)[1:]+list(train_peptides_pivot.columns)[1:]
features_name1=["have_%s"%i for i in valid_month]+["month_%s"%i for i in valid_month]+label_4_name
features_name2=["have_%s"%i for i in valid_month]+["month_%s"%i for i in valid_month]+label_3_name
features_name3=["have_%s"%i for i in valid_month]+["month_%s"%i for i in valid_month]+label_2_name
features_name4=["have_%s"%i for i in valid_month]+["month_%s"%i for i in valid_month]
features_name5=features_name1+p_features
history_month_dic={}


model_version_path="/kaggle/input/amp-parkinson/models_ensemble/label1/"
model1_list=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name1,1)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model1_list.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble/label4/"
model2_list=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name2,4)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model2_list.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble/time4/"
model3_list=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name3,4)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model3_list.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble/label16/"
model4_list=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name4,16)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model4_list.append(model)
    
# model_version_path="/kaggle/input/amp-parkinson/models_seq2/"
# model_list5=[]
# for w in os.listdir(model_version_path):
#     model=MLPModel(features_name5,1)
#     model.cuda()
#     model.load_state_dict(torch.load(model_version_path+w))
#     model_list5.append(model)
    
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble2/label1/"
model1_list2=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name1+p_features,1)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model1_list2.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble2/label4/"
model2_list2=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name2+p_features,4)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model2_list2.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble2/time4/"
model3_list2=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name3+p_features,4)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model3_list2.append(model)
    
model_version_path="/kaggle/input/amp-parkinson/models_ensemble2/label16/"
model4_list2=[]
for w in os.listdir(model_version_path):
    model=MLPModel(features_name4+p_features,16)
    model.cuda()
    model.load_state_dict(torch.load(model_version_path+w))
    model4_list2.append(model)
    
    
def have_month(m,p,i,dic):
    if i>m:
        return -1
    else:
        if i in dic.get(p,[]):
            return 1
        else:
            return 0
        

In [ ]:
def get_pred(df_test, df_test_peptides, df_test_proteins, df_submission):
    df_submission['patient_id'] = df_submission.apply('prediction_id').str.split('_', expand=True)[0].astype(int)
    df_submission['visit_month'] = df_submission.apply('prediction_id').str.split('_', expand=True)[1].astype(int)
    df_submission['visit_id']=df_submission['patient_id'].astype(str)+"_"+df_submission['visit_month'].astype(str)
    
    
    all_data=df_submission[["patient_id","visit_month","visit_id"]].drop_duplicates().reset_index(drop=True)
    for row in all_data.itertuples():
        if row.patient_id not in history_month_dic.keys():
            history_month_dic[row.patient_id]=[row.visit_month]
        else:
            history_month_dic[row.patient_id].append(row.visit_month)

    #1
    all_data1=all_data.merge(pd.DataFrame({"out_type":label_4_name}),how="cross")
    for i in valid_month:
        all_data1["have_%s"%i]=list(map(lambda m,p:have_month(m,p,i,history_month_dic),all_data1["visit_month"],all_data1["patient_id"]))
        all_data1["month_%s"%i]=(all_data1["visit_month"]==i).astype(int)

    for i in label_4_name:
        all_data1[i]=(all_data1["out_type"]==i).astype(int)

    #2
    all_data2=all_data.merge(pd.DataFrame({"out_type":label_3_name}),how="cross")
    for i in valid_month:
        all_data2["have_%s"%i]=list(map(lambda m,p:have_month(m,p,i,history_month_dic),all_data2["visit_month"],all_data2["patient_id"]))
        all_data2["month_%s"%i]=(all_data2["visit_month"]==i).astype(int)

    for i in label_3_name:
        all_data2[i]=(all_data2["out_type"]==i).astype(int)

    #3
    all_data3=all_data.merge(pd.DataFrame({"out_type":label_2_name}),how="cross")
    for i in valid_month:
        all_data3["have_%s"%i]=list(map(lambda m,p:have_month(m,p,i,history_month_dic),all_data3["visit_month"],all_data3["patient_id"]))
        all_data3["month_%s"%i]=(all_data3["visit_month"]==i).astype(int)

    for i in label_2_name:
        all_data3[i]=(all_data3["out_type"]==i).astype(int)

    #4
    all_data4=all_data.copy()
    for i in valid_month:
        all_data4["have_%s"%i]=list(map(lambda m,p:have_month(m,p,i,history_month_dic),all_data4["visit_month"],all_data4["patient_id"]))
        all_data4["month_%s"%i]=(all_data4["visit_month"]==i).astype(int)  
        
        
    #5
#     all_data=all_data.merge(pd.DataFrame({"out_type":label_4_name}),how="cross")
#     for i in valid_month:
#         all_data["have_%s"%i]=list(map(lambda m,p:have_month_ori(m,p,i),all_data["visit_month"],all_data["patient_id"]))
#         all_data["month_%s"%i]=(all_data["visit_month"]==i).astype(int)

#     for i in label_4_name:
#         all_data[i]=(all_data["out_type"]==i).astype(int)
        

    train_peptides=Peptide.merge(df_test_peptides,on="Peptide",how="left").fillna(0)
    train_proteins_pivot=NPX.merge(df_test_proteins,on="NPX",how="left").fillna(0)

    train_peptides["PeptideAbundance"]=train_peptides["PeptideAbundance"]/(train_peptides.groupby("visit_id").PeptideAbundance.transform("sum")+1)
    train_proteins["NPX"]=train_proteins["NPX"]/(train_proteins.groupby("visit_id").NPX.transform("sum")+1)

    train_proteins_pivot=train_proteins.pivot(index=["visit_id"],columns=["UniProt"],values=["NPX"]).reset_index()
    train_proteins_pivot.columns=["_".join(i).strip("_") for i in train_proteins_pivot.columns]

    train_peptides_pivot=train_peptides.pivot(index=["visit_id"],columns=["Peptide"],values=["PeptideAbundance"]).reset_index()
    train_peptides_pivot.columns=["_".join(i).strip("_") for i in train_peptides_pivot.columns]
    
    #all_data=all_data.merge(train_proteins_pivot,on="visit_id",how="left").merge(train_peptides_pivot,on="visit_id",how="left").fillna(0)

    pred_list=[]
    pred_list2=[]
    with torch.no_grad():
        for model in model1_list:
            model.eval()
            pred = model(torch.Tensor(all_data1[features_name1].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)
        for model in model2_list:
            model.eval()
            pred = model(torch.Tensor(all_data2[features_name2].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1,4,4)
            pred=np.concatenate([pred[i,:] for i in range(len(pred))],axis=1).T.reshape(-1)
            pred_list.append(pred)
        for model in model3_list:
            model.eval()
            pred = model(torch.Tensor(all_data3[features_name3].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)
        for model in model4_list:
            model.eval()
            pred = model(torch.Tensor(all_data4[features_name4].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)


        for model in model1_list2:
            model.eval()
            pred = model(torch.Tensor(all_data1.merge(train_proteins_pivot,on="visit_id",how="left").merge(train_peptides_pivot,on="visit_id",how="left").fillna(0)[features_name1+p_features].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)
        for model in model2_list2:
            model.eval()
            pred = model(torch.Tensor(all_data2.merge(train_proteins_pivot,on="visit_id",how="left").merge(train_peptides_pivot,on="visit_id",how="left").fillna(0)[features_name2+p_features].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1,4,4)
            pred=np.concatenate([pred[i,:] for i in range(len(pred))],axis=1).T.reshape(-1)
            pred_list2.append(pred)
        for model in model3_list2:
            model.eval()
            pred = model(torch.Tensor(all_data3.merge(train_proteins_pivot,on="visit_id",how="left").merge(train_peptides_pivot,on="visit_id",how="left").fillna(0)[features_name3+p_features].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)
        for model in model4_list2:
            model.eval()
            pred = model(torch.Tensor(all_data4.merge(train_proteins_pivot,on="visit_id",how="left").merge(train_peptides_pivot,on="visit_id",how="left").fillna(0)[features_name4+p_features].values).cuda())
            pred=pred.cpu().detach().numpy().reshape(-1)
            pred_list.append(pred)

    pred_avg=np.mean(np.array(pred_list),axis=0)
    pred_avg[pred_avg<0]=0
    pred_avg2=np.mean(np.array(pred_list2),axis=0)
    pred_avg2[pred_avg2<0]=0 
    
    return pred_avg,pred_avg2

In [ ]:
import amp_pd_peptide

env = amp_pd_peptide.make_env()
test_iterator = env.iter_test()
        

for (df_test, df_test_peptides, df_test_proteins, df_submission) in test_iterator:
    pred_avg,pred_avg2=get_pred(df_test, df_test_peptides, df_test_proteins, df_submission)
    
    df_submission["rating1"]=pred_avg
    df_submission["rating2"]=pred_avg2
    
    df_submission["rating"]=list(map(lambda a,b,c:a*0.45+b*0.55 if c in [0,6] else a*0.55+b*0.45,df_submission["rating1"],df_submission["rating2"],df_submission["visit_month"]))
    df_submission["rating"]=df_submission["rating"].apply(lambda x:np.round(x,0))
    
    df_submission = df_submission.loc[:, ['prediction_id', 'rating']]
    env.predict(df_submission)

In [ ]:
df_submission